<a href="https://colab.research.google.com/github/NoahLee99/ML-DL-studylog/blob/main/%EC%98%A4%EC%B0%A8%20%EC%97%AD%EC%A0%84%ED%8C%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **오차 역전파**

* 한 번의 "순전파(forward propagtion)"가 일어나면 각 층의 가중치 초깃값이 정해진다.
<br> 이 초깃값의 가중치로 만들어진 값(출력 값)과 실제 값을 비교해 출력층의 오차를 계산한다.

* 이때 계산된 **출력층의 오차를 최소화시키기 위해**
<br> 출력층에서 입력층으로 거꾸로 전파하여 <br> 각 층의 가중치(weight)와 편향(bias)을 수정(업데이트)하는
<br> 알고리즘이 바로 **"역전파(backward propagation)"**이다.

* 역전파는 **"연쇄 법칙(chain rule)"**을 기반으로 하며,
<br> "손실 함수(loss function)"의 기울기를 계산하여
<br> 가중치가 손실 함수의 변화에 따라 어떻게 변화해야 하는지를 결정한다.

* 역전파에 한계점이 존재하는데,
<br> 출력층에서 시작된 가중치 업데이트가 처음 층까지 전달되지 않는 현상이 생기는 문제가 있다.
<br> 이는 활성화 함수로 사용된 시그모이드 함수의 특성 때문이다.
<br> 시그모이드(0~1 사이의 값만 존재) 함수를 미분하면 최대치는 0.25이다.
<br> 시그모이드 함수의 값들은 1보다 작으므로 계속 곱하다 보면 0에 가까워진다.
<br> 따라서 여러 층을 거칠수록 기울기가 사라지는 **"기울기 소실(vanishing gradient)"** 문제가 발생한다.
<br> 이를 해결하고자 **"렐루(ReLU)"**라는 활성화 함수가 탄생했다.

* 렐루는 x가 0보다 작을 때 모든 값을 0으로 처리하고,
<br> 0보다 큰 값은 x를 그대로 사용하는 방법이다.
<br> 따라서 x가 0보다 크기만 하면 미분 값은 1이 되어 기울기 소실 문제를 해결할 수 있다.

In [1]:
# XOR 문제를 사용해 오차 역전파를 파이썬 코드로 확인하고자 한다.

# 환경 변수 설정하기

In [2]:
# 실제 값이 될 XOR 진리표
# x1 = [0, 0, 1, 1]
# x2 = [0, 1, 0, 1]
# 결과값 = [0, 1, 1, 0]

In [1]:
# XOR 진리표를 두개의 입력 값과 한 개의 타깃 값으로 설정

# 입력 값 및 타깃 값
import random
import numpy as np

data = [
    [[0, 0], [0]],
    [[0, 1], [1]],
    [[1, 0], [1]],
    [[1, 1], [0]]
]

# 실행 횟수(iterations), 학습률(lr), 모멘텀 계수(mo) 설정
iterations=5000
lr=0.1
mo=0.4

In [2]:
# 활성화 함수 지정 - 1.시그모이드

# 미분할 때와 아닐 때의 각각의 값
def sigmoid(x, derivative=False):
    if (derivative == True):
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

# 활성화 함수 지정 - 2.tanh
# tanh 함수의 미분은 1 - (활성화 함수 출력의 제곱)
def tanh(x, derivative=False):
    if (derivative == True):
        return 1 - x ** 2
    return np.tanh(x)

# 가중치 배열 만드는 함수
def makeMatrix(i, j, fill=0.0):
    mat = []
    for i in range(i):
        mat.append([fill] * j)
    return mat

# 파이썬 코드로 실행하는 신경망

In [5]:
# 신경망의 실행
class NeuralNetwork:

    # 초깃값의 지정
    def __init__(self, num_x, num_yh, num_yo, bias=1):

        # 입력값(num_x), 은닉층 초깃값(num_yh), 출력층 초깃값(num_yo), 바이어스
        self.num_x = num_x + bias  # 바이어스는 1로 지정(본문 참조)
        self.num_yh = num_yh
        self.num_yo = num_yo

        # 활성화 함수 초깃값
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_yo

        # 가중치 입력 초깃값
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()

        # 가중치 출력 초깃값
        self.weight_out = makeMatrix(self.num_yh, self.num_yo)
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                self.weight_out[j][k] = random.random()

        # 모멘텀 SGD를 위한 이전 가중치 초깃값
        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_yo)

    # 업데이트 함수
    def update(self, inputs):

        # 입력 레이어의 활성화 함수
        for i in range(self.num_x - 1):
            self.activation_input[i] = inputs[i]

        # 은닉층의 활성화 함수
        for j in range(self.num_yh):
            sum = 0.0
            for i in range(self.num_x):
                sum = sum + self.activation_input[i] * self.weight_in[i][j]
            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_hidden[j] = tanh(sum, False)

        # 출력층의 활성화 함수
        for k in range(self.num_yo):
            sum = 0.0
            for j in range(self.num_yh):
                sum = sum + self.activation_hidden[j] * self.weight_out[j][k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_out[k] = tanh(sum, False)

        return self.activation_out[:]

    # 역전파의 실행
    def backPropagate(self, targets):

        # 델타 출력 계산
        output_deltas = [0.0] * self.num_yo
        for k in range(self.num_yo):
            error = targets[k] - self.activation_out[k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            output_deltas[k] = tanh(self.activation_out[k], True) * error

        # 은닉 노드의 오차 함수
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_yo):
                error = error + output_deltas[k] * self.weight_out[j][k]
                # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error

        # 출력 가중치 업데이트
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo * self.gradient_out[j][k] - lr * gradient
                self.weight_out[j][k] += v
                self.gradient_out[j][k] = gradient

        # 입력 가중치 업데이트
        for i in range(self.num_x):
            for j in range(self.num_yh):
                gradient = hidden_deltas[j] * self.activation_input[i]
                v = mo*self.gradient_in[i][j] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient

        # 오차의 계산(최소 제곱법)
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5 * (targets[k] - self.activation_out[k]) ** 2
        return error

    # 학습 실행
    def train(self, patterns):
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets)
            if i % 500 == 0:
                print('error: %-.5f' % error)
    # 결괏값 출력
    def result(self, patterns):
        for p in patterns:
            print('Input: %s, Predict: %s' % (p[0], self.update(p[0])))

if __name__ == '__main__':

    # 두 개의 입력 값, 두 개의 레이어, 하나의 출력 값을 갖도록 설정
    n = NeuralNetwork(2, 2, 1)

    # 학습 실행
    n.train(data)

    # 결괏값 출력
    n.result(data)


# Reference: http://arctrix.com/nas/python/bpnn.py (Neil Schemenauer)

error: 0.63006
error: 0.04397
error: 0.00240
error: 0.00088
error: 0.00051
error: 0.00035
error: 0.00027
error: 0.00021
error: 0.00018
error: 0.00015
Input: [0, 0], Predict: [0.0010564190342788123]
Input: [0, 1], Predict: [0.9885356741580501]
Input: [1, 0], Predict: [0.9885588149855168]
Input: [1, 1], Predict: [0.002302951994419993]
